In [1]:
import numpy as np
import pandas as pd

file = (r'C:/Users/ninap/enwiki_4.csv')

df = pd.read_csv(file, sep = '#', quotechar='|', index_col = False, engine = 'python')
df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%dT%H:%M:%SZ')

In [2]:
df.head()

,page_id,page_title,page_ns,revision_id,timestamp,contributor_id,contributor_name,bytes,model
0,10,AccessibleComputing,0,233192,2001-01-21 02:12:21,99,RoseParks,124,wikitext
1,10,AccessibleComputing,0,862220,2002-02-25 15:43:11,1226483,Conversion script,35,wikitext
2,10,AccessibleComputing,0,15898945,2003-04-25 22:18:38,7543,Ams80,34,wikitext
3,10,AccessibleComputing,0,56681914,2006-06-03 16:55:41,516514,Nzd,36,wikitext
4,10,AccessibleComputing,0,74466685,2006-09-08 04:16:04,750223,Rory096,57,wikitext


In [3]:
# entfällt, wenn parser angepasst!

df = df.drop(['page_ns', 'model'], axis=1)
df.head()

,page_id,page_title,revision_id,timestamp,contributor_id,contributor_name,bytes
0,10,AccessibleComputing,233192,2001-01-21 02:12:21,99,RoseParks,124
1,10,AccessibleComputing,862220,2002-02-25 15:43:11,1226483,Conversion script,35
2,10,AccessibleComputing,15898945,2003-04-25 22:18:38,7543,Ams80,34
3,10,AccessibleComputing,56681914,2006-06-03 16:55:41,516514,Nzd,36
4,10,AccessibleComputing,74466685,2006-09-08 04:16:04,750223,Rory096,57


# Abfragen auf Basis pro Artikel

**Wie viele Revisionen wurden vorgenommen?**

Abfrage zählt die Anzahl der einzigartigen Revision-IDs pro Artikel und gibt zusätzlich die durchschnittliche Größe der Revisionen in bytes pro Artikel an. 

In [22]:
revcount = df.groupby(['page_title']).agg({'revision_id' : 'count', 'bytes' : 'mean'})
revcount = revcount.rename({'revision_id' : 'revcount', 'bytes' : 'meansize'}, axis = "columns")
revcount.sort_values(['revcount'], ascending=False)

,revcount,meansize
page_title,,
Anarchism,19961,84662.829167
Abraham Lincoln,17613,108672.676375
Albert Einstein,17476,86462.335031
Albania,16271,94737.645873
American Revolutionary War,15881,134133.046597
...,...,...
Abbreviations,4,7286.500000
AbensbergGermany,4,224.000000
AutoMorphism,4,197.500000


**Wie viele Revisionen wurden je Datum vorgenommen?**

Schritt 1: Neue Spalten "dates" und "times" generieren. 

In [5]:
df['dates'] = df['timestamp'].dt.date
df['times'] = df['timestamp'].dt.time
df.head()

,page_id,page_title,revision_id,timestamp,contributor_id,contributor_name,bytes,dates,times
0,10,AccessibleComputing,233192,2001-01-21 02:12:21,99,RoseParks,124,2001-01-21,02:12:21
1,10,AccessibleComputing,862220,2002-02-25 15:43:11,1226483,Conversion script,35,2002-02-25,15:43:11
2,10,AccessibleComputing,15898945,2003-04-25 22:18:38,7543,Ams80,34,2003-04-25,22:18:38
3,10,AccessibleComputing,56681914,2006-06-03 16:55:41,516514,Nzd,36,2006-06-03,16:55:41
4,10,AccessibleComputing,74466685,2006-09-08 04:16:04,750223,Rory096,57,2006-09-08,04:16:04


Schritt 2: Abfrage gruppiert die Ausgabe nach Titel und Datum und zeigt an, wie viele RevidentInnen an diesem Tag mit durchschnittlich wie vielen Bytes beigetragen haben. 

In [24]:
revdate = df.groupby(['page_title', 'dates']).agg({'revision_id' : 'count', 'bytes' : 'mean'})
revdate = revdate.rename({'revision_id' : 'revcount', 'bytes' : 'meansize'}, axis = "columns")
revdate.sort_values(['page_title', 'dates'], ascending = True)

revcount  meansize
page_title                                     dates                         
A                                              2001-05-17         1    6784.0
                                               2001-06-26         1     871.0
                                               2001-07-05         2    7374.0
                                               2002-02-02         1    7528.0
                                               2002-02-25         1    7565.0
...                                                             ...       ...
Wikipedia:Adding Wikipedia articles to Nupedia 2002-12-20         2    6464.0
                                               2003-01-10         1    6667.0
                                               2003-02-07         1    6596.0
                                               2003-03-12         1    6690.0
                                               2003-03-17         1      45.0

[179720 rows x 2 columns]

**Revisionen pro Datum kumuliert** 

Schritt 1: neue Spalte "revdates" generieren: gibt die Anzahl der Revisionen für diesen Artikel an diesem Tag an.

In [7]:
df['revdates'] = df.groupby(by='dates')['dates'].transform('count')
df

,page_id,page_title,revision_id,timestamp,contributor_id,contributor_name,bytes,dates,times,revdates
0,10,AccessibleComputing,233192,2001-01-21 02:12:21,99,RoseParks,124,2001-01-21,02:12:21,7
1,10,AccessibleComputing,862220,2002-02-25 15:43:11,1226483,Conversion script,35,2002-02-25,15:43:11,269
2,10,AccessibleComputing,15898945,2003-04-25 22:18:38,7543,Ams80,34,2003-04-25,22:18:38,8
3,10,AccessibleComputing,56681914,2006-06-03 16:55:41,516514,Nzd,36,2006-06-03,16:55:41,161
4,10,AccessibleComputing,74466685,2006-09-08 04:16:04,750223,Rory096,57,2006-09-08,04:16:04,300
...,...,...,...,...,...,...,...,...,...,...
548300,857,Aberdeenshire,1063762069,2022-01-04 19:47:38,42968770,ArticleMaker2,34041,2022-01-04,19:47:38,36
548301,857,Aberdeenshire,1063898137,2022-01-05 13:51:33,10291445,Wiki Gh!,34042,2022-01-05,13:51:33,72
548302,857,Aberdeenshire,1064618213,2022-01-09 09:19:28,42968770,ArticleMaker2,34201,2022-01-09,09:19:28,24
548303,857,Aberdeenshire,1065536710,2022-01-14 02:29:34,42425010,Rlink2,34255,2022-01-14,02:29:34,44


Schritt 2: Abfrage generiert die kumulierte Summe der täglichen Revisionszahlen auf Basis des Artikels und des Datums

In [8]:
revcums = df.groupby(['page_title', 'dates']).sum().groupby(level=0).cumsum().reset_index()
revcums.head()

,page_title,dates,page_id,revision_id,bytes,revdates
0,A,2001-05-17,290,263232,6784,2
1,A,2001-06-26,580,466368321,7655,3
2,A,2001-07-05,1160,932706906,22403,7
3,A,2002-02-02,1450,932710363,29931,11
4,A,2002-02-25,1740,932795893,37496,280


**Wie viele Revisionen wurden von den einzelnen RevidentInnen bearbeitet?**

Abfrage gruppiert nach Artikel, Username und Datum, zählt die Revisionen an diesem Tag und gibt die durchschnittliche Revisionsgröße aus.

In [32]:
revuser = df.groupby(['page_title', 'contributor_name', 'contributor_id', 'dates']).agg({'revdates' : 'count', 'bytes' : 'mean'})
revuser.sort_values(['page_title', 'revdates'], ascending = False)

# TODO: kontrollieren, ob anonyme eh einzeln (je id) ausgegeben werden



revdates  \
page_title                                     contributor_name              contributor_id dates                  
Wikipedia:Adding Wikipedia articles to Nupedia Anonymous                     211.26.134.83  2002-12-20         2   
                                               Conversion script             1226483        2002-02-25         1   
                                               David Martland                5457           2002-11-18         1   
                                               Eclecticology                 372            2002-05-19         1   
                                               Larry_Sanger                  0              2001-03-28         1   
...                                                                                                          ...   
A                                              Å½iedas                       8859014        2010-01-05         1   
                                               Ð¡Ð°ÑˆÐ° Ð¡Ñ‚ÐµÑ„Ð°Ð½Ð¾Ð²Ð¸Ñ› 205156         2005-11-22         1   
                                               Ð£Ñ‡Ñ…Ñ™Ñ‘Ð½Ð°Ñ�              30241636       2018-02-19         1   
                                                                                            2018-03-13         1   
                                               ã�¨ã�‚ã‚‹ç™½ã�„çŒ«            184109         2005-11-04         1   

                                                                                                          bytes  
page_title                                     contributor_name              contributor_id dates                
Wikipedia:Adding Wikipedia articles to Nupedia Anonymous                     211.26.134.83  2002-12-20   6464.0  
                                               Conversion script             1226483        2002-02-25   5059.0  
                                               David Martland                5457           2002-11-18   6457.0  
                                               Eclecticology                 372            2002-05-19   5006.0  
                                               Larry_Sanger                  0              2001-03-28   4637.0  
...                                                                                                         ...  
A                                              Å½iedas                       8859014        2010-01-05   8649.0  
                                               Ð¡Ð°ÑˆÐ° Ð¡Ñ‚ÐµÑ„Ð°Ð½Ð¾Ð²Ð¸Ñ› 205156         2005-11-22   9942.0  
                                               Ð£Ñ‡Ñ…Ñ™Ñ‘Ð½Ð°Ñ�              30241636       2018-02-19  22639.0  
                                                                                            2018-03-13  21315.0  
                                               ã�¨ã�‚ã‚‹ç™½ã�„çŒ«            184109         2005-11-04   9939.0  

[369149 rows x 2 columns]

In [40]:
revuser1 = revuser.reset_index()
filter = ((revuser1['contributor_name'].str.contains('anonymous', case = False)) & (revuser1['page_title'].str.contains('anarch', case = False)))
revuser1[filter]['contributor_name']

205372           Anonymous
205373           Anonymous
205374           Anonymous
205375           Anonymous
205376           Anonymous
                ...       
207508           Anonymous
207509           Anonymous
207510           Anonymous
207511           Anonymous
207512    Anonymous editor
Name: contributor_name, Length: 2141, dtype: object

**Welche UserInnen verändern in ihren Revisionen das meiste (im Vergleich zum Schnitt)?**

Schritt 1: Spalte "revbytes" generieren zur Ausgabe der Summe der veränderten Bytes pro Artikel

In [10]:
df['revbytes'] = df.groupby(by='page_title')['bytes'].transform('sum')
df

,page_id,page_title,revision_id,timestamp,contributor_id,contributor_name,bytes,dates,times,revdates,revbytes
0,10,AccessibleComputing,233192,2001-01-21 02:12:21,99,RoseParks,124,2001-01-21,02:12:21,7,2848
1,10,AccessibleComputing,862220,2002-02-25 15:43:11,1226483,Conversion script,35,2002-02-25,15:43:11,269,2848
2,10,AccessibleComputing,15898945,2003-04-25 22:18:38,7543,Ams80,34,2003-04-25,22:18:38,8,2848
3,10,AccessibleComputing,56681914,2006-06-03 16:55:41,516514,Nzd,36,2006-06-03,16:55:41,161,2848
4,10,AccessibleComputing,74466685,2006-09-08 04:16:04,750223,Rory096,57,2006-09-08,04:16:04,300,2848
...,...,...,...,...,...,...,...,...,...,...,...
548300,857,Aberdeenshire,1063762069,2022-01-04 19:47:38,42968770,ArticleMaker2,34041,2022-01-04,19:47:38,36,14056754
548301,857,Aberdeenshire,1063898137,2022-01-05 13:51:33,10291445,Wiki Gh!,34042,2022-01-05,13:51:33,72,14056754
548302,857,Aberdeenshire,1064618213,2022-01-09 09:19:28,42968770,ArticleMaker2,34201,2022-01-09,09:19:28,24,14056754
548303,857,Aberdeenshire,1065536710,2022-01-14 02:29:34,42425010,Rlink2,34255,2022-01-14,02:29:34,44,14056754


Schritt 2: Abfrage der veränderten Bytes pro Artikel je User und Gegenüberstellung durch Ausgabe der Summe aller Revisionsbytes des Artikels

In [11]:
revuserbytes = df.groupby(['page_title', 'contributor_name', 'bytes']).agg({'revdates' : 'count', 'revbytes' : 'sum'})
revuserbytes.sort_values(['page_title', 'bytes'], ascending = False)

revdates  \
page_title                                     contributor_name bytes             
Wikipedia:Adding Wikipedia articles to Nupedia Stephen Gilbert  6690          1   
                                               MartinHarper     6667          1   
                                                                6596          1   
                                               Anonymous        6471          1   
                                                                6457          1   
...                                                                         ...   
A                                              Matt1314         0             1   
                                               Mattslick1       0             2   
                                               NanobyteExo      0             1   
                                               Starman1985      0             7   
                                               Zalgt1           0             1   

                                                                        revbytes  
page_title                                     contributor_name bytes             
Wikipedia:Adding Wikipedia articles to Nupedia Stephen Gilbert  6690       54085  
                                               MartinHarper     6667       54085  
                                                                6596       54085  
                                               Anonymous        6471       54085  
                                                                6457       54085  
...                                                                          ...  
A                                              Matt1314         0       47173800  
                                               Mattslick1       0       94347600  
                                               NanobyteExo      0       47173800  
                                               Starman1985      0      330216600  
                                               Zalgt1           0       47173800  

[493278 rows x 2 columns]

**Wie groß sind die Revisionen im Verhältnis?**

Schritt 1: Generieren der Spalte "revperc", die ausgibt, welchen prozentuellen Anteil die Revision an der Gesamtzahl hat

In [12]:
df['revperc'] = 100/df['revbytes']*df['bytes']
df

,page_id,page_title,revision_id,timestamp,contributor_id,contributor_name,bytes,dates,times,revdates,revbytes,revperc
0,10,AccessibleComputing,233192,2001-01-21 02:12:21,99,RoseParks,124,2001-01-21,02:12:21,7,2848,4.353933
1,10,AccessibleComputing,862220,2002-02-25 15:43:11,1226483,Conversion script,35,2002-02-25,15:43:11,269,2848,1.228933
2,10,AccessibleComputing,15898945,2003-04-25 22:18:38,7543,Ams80,34,2003-04-25,22:18:38,8,2848,1.193820
3,10,AccessibleComputing,56681914,2006-06-03 16:55:41,516514,Nzd,36,2006-06-03,16:55:41,161,2848,1.264045
4,10,AccessibleComputing,74466685,2006-09-08 04:16:04,750223,Rory096,57,2006-09-08,04:16:04,300,2848,2.001404
...,...,...,...,...,...,...,...,...,...,...,...,...
548300,857,Aberdeenshire,1063762069,2022-01-04 19:47:38,42968770,ArticleMaker2,34041,2022-01-04,19:47:38,36,14056754,0.242168
548301,857,Aberdeenshire,1063898137,2022-01-05 13:51:33,10291445,Wiki Gh!,34042,2022-01-05,13:51:33,72,14056754,0.242175
548302,857,Aberdeenshire,1064618213,2022-01-09 09:19:28,42968770,ArticleMaker2,34201,2022-01-09,09:19:28,24,14056754,0.243307
548303,857,Aberdeenshire,1065536710,2022-01-14 02:29:34,42425010,Rlink2,34255,2022-01-14,02:29:34,44,14056754,0.243691


Schritt 2: Abfrage sortiert nach Artikel, RevidentInnen und prozentuellem Revisionsanteil

In [13]:
revsize = df.groupby(['page_title', 'contributor_name', 'revperc']).agg({'revdates' : 'count', 'revbytes' : 'sum'})
revsize.sort_values(['page_title', 'contributor_name', 'revperc'], ascending = False)

revdates  \
page_title                                     contributor_name revperc               
Wikipedia:Adding Wikipedia articles to Nupedia Stephen Gilbert  12.369419         1   
                                               MartinHarper     12.326893         1   
                                                                12.195618         1   
                                                                0.083202          1   
                                               Larry_Sanger     8.573542          1   
...                                                                             ...   
A                                              1029384756pie    0.017007          1   
                                                                0.000040          2   
                                               .:Ajvol:.        0.018154          1   
                                               -Midorihana-     0.015242          1   
                                               *Kat*            0.019430          1   

                                                                           revbytes  
page_title                                     contributor_name revperc              
Wikipedia:Adding Wikipedia articles to Nupedia Stephen Gilbert  12.369419     54085  
                                               MartinHarper     12.326893     54085  
                                                                12.195618     54085  
                                                                0.083202      54085  
                                               Larry_Sanger     8.573542      54085  
...                                                                             ...  
A                                              1029384756pie    0.017007   47173800  
                                                                0.000040   94347600  
                                               .:Ajvol:.        0.018154   47173800  
                                               -Midorihana-     0.015242   47173800  
                                               *Kat*            0.019430   47173800  

[493278 rows x 2 columns]

**Was sind die Maximalwerte bei den Revisionen?**

Abfrage nach Datum und Sortierung nach Artikel und absteigend sortiert nach Revisionsgröße

In [14]:
revspikes = df.groupby(['page_title', 'dates']).agg({'bytes' : 'max'})
revspikes.sort_values(['page_title', 'bytes', 'dates'], ascending = False)

bytes
page_title                                     dates            
Wikipedia:Adding Wikipedia articles to Nupedia 2003-03-12   6690
                                               2003-01-10   6667
                                               2003-02-07   6596
                                               2002-12-20   6471
                                               2002-11-18   6457
...                                                          ...
A                                              2009-04-24   5464
                                               2009-04-23   5464
                                               2009-04-22   5403
                                               2001-06-26    871
                                               2004-05-28    687

[179720 rows x 1 columns]

**Machen UserInnen, die häufiger revidieren auch größere Revisionen?**

Schritt 1: Generieren der Spalte "nrevs", die die Anzahl der Revisionen pro UserIn zählt

In [15]:
df['nrevs'] = df.groupby(by='contributor_id')['contributor_id'].transform('count')
df

,page_id,page_title,revision_id,timestamp,contributor_id,contributor_name,bytes,dates,times,revdates,revbytes,revperc,nrevs
0,10,AccessibleComputing,233192,2001-01-21 02:12:21,99,RoseParks,124,2001-01-21,02:12:21,7,2848,4.353933,8
1,10,AccessibleComputing,862220,2002-02-25 15:43:11,1226483,Conversion script,35,2002-02-25,15:43:11,269,2848,1.228933,267
2,10,AccessibleComputing,15898945,2003-04-25 22:18:38,7543,Ams80,34,2003-04-25,22:18:38,8,2848,1.193820,16
3,10,AccessibleComputing,56681914,2006-06-03 16:55:41,516514,Nzd,36,2006-06-03,16:55:41,161,2848,1.264045,2
4,10,AccessibleComputing,74466685,2006-09-08 04:16:04,750223,Rory096,57,2006-09-08,04:16:04,300,2848,2.001404,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548300,857,Aberdeenshire,1063762069,2022-01-04 19:47:38,42968770,ArticleMaker2,34041,2022-01-04,19:47:38,36,14056754,0.242168,2
548301,857,Aberdeenshire,1063898137,2022-01-05 13:51:33,10291445,Wiki Gh!,34042,2022-01-05,13:51:33,72,14056754,0.242175,40
548302,857,Aberdeenshire,1064618213,2022-01-09 09:19:28,42968770,ArticleMaker2,34201,2022-01-09,09:19:28,24,14056754,0.243307,2
548303,857,Aberdeenshire,1065536710,2022-01-14 02:29:34,42425010,Rlink2,34255,2022-01-14,02:29:34,44,14056754,0.243691,83


Abfrage der Maximalwerte bei der Anzahl der Revisionen pro UserIn und Angabe der Summe an Bytes der Revisionen

In [16]:
revmaxuser = df.groupby(['page_title', 'contributor_id']).agg({'nrevs' : 'max', 'bytes' : 'sum'})
revmaxuser.sort_values(['page_title', 'nrevs', 'bytes'], ascending = False)

nrevs  bytes
page_title                                     contributor_id              
Wikipedia:Adding Wikipedia articles to Nupedia 1226483           267   5059
                                               0                 244   4637
                                               372               138   5006
                                               5862               23  13308
                                               86                 14   6690
...                                                              ...    ...
A                                              88.110.255.25       1      0
                                               88.254.117.198      1      0
                                               90.193.250.143      1      0
                                               98.230.29.136       1      0
                                               99.129.141.193      1      0

[232677 rows x 2 columns]

**Wie ist das Verhältnis von anonymen RevidentInnen zu namentlich genannten?**

In [17]:
filter1 = (df['contributor_name'].str.contains('anonym', case = False))
revanon = len(df[filter1]['nrevs'])
diff = 100/len(df)*revanon
revpropnames = print('named: ', round(100-diff, 2), '% \n', 'anon: ', round(diff, 2), '%')
revpropnames

named:  68.53 % 
 anon:  31.47 %
